In [1]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
import datetime
import os
current_time = datetime.datetime.now().date()
if current_time< datetime.date(2024, 1, 1):
    llm_model="The current date is before January 1, 2024. Exiting."
else:
    llm_model="model is ollma mistral"
    
print(llm_model)
    


model is ollma mistral


In [11]:
from langchain.document_loaders import PyPDFLoader
docs = PyPDFLoader("research.pdf").load()


In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
split_docs = text_splitter.split_documents(docs)

In [15]:
split_docs

[Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-08-22T12:54:44+05:45', 'author': 'Guptaraj Shrestha, Nishan Thing, Prajita Dhakal, Prasanga Dahal, Pukar Karki, Manoj Kumar Guragain', 'title': 'Nepali Sign Language Letter Detection and Finger Spelling Using Mediapipe and CNN', 'subject': '14th IOE Graduate Conference', 'keywords': 'Nepali Sign Language, Machine Learning, Convolutional Neural Network, MediaPipe', 'moddate': '2024-08-22T12:54:44+05:45', 'trapped': '/False', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'source': 'research.pdf', 'total_pages': 5, 'page': 0, 'page_label': '166'}, page_content='Proceedings of 15th IOE Graduate Conference\nPeer Reviewed\nYear: 2024 Month: May Volume: 15\nISSN: 2350-8914 (Online), 2350-8906 (Print)\nNepali Sign Language Letter Detection and Finger Spelling Using\nMediapipe and CNN\nGuptaraj Shrestha a, Nishan Thing b, Prajit

In [20]:
from langchain.vectorstores import Chroma
persist_dic="QA/chroma"
model= OllamaEmbeddings(model="mistral")
vectordb = Chroma(
    embedding_function=model,
    persist_directory=persist_dic,
    )

In [21]:
vectordb.add_documents(split_docs)

['26d1f4aa-0df1-4588-be34-830a9d1b0bff',
 '7e1e715d-fe7a-45f9-8b59-4eb838363009',
 '738b844b-77fa-4fc2-b2b4-2c78234c8b66',
 'b83aee48-6447-4c88-bb34-047140fa1caa',
 '325358e2-39e9-4068-bf31-2802952397ea',
 '5305e133-a306-4824-8881-9c56f873b332',
 '677ad2a4-7146-4cd9-993e-26000a07ddb8',
 '1b4bb9d9-3a96-4b54-ad94-06b3903849d0',
 '34c1f4ef-3ddc-4984-b9f6-af093c6bc956',
 '86908de8-f04e-404d-8ffa-66ac8340d0ca',
 '549aa21e-3656-4f0d-9cc1-80725ab9b9b2',
 '4531616a-ef1c-4efe-9dcc-21d48dafd19d',
 'acc67585-95dc-42c3-a582-19f547f2732f',
 'fc5e6f91-3096-4c90-bb78-a1c4052b9f94',
 '844dc923-777d-4777-b805-6a0984e9bd45',
 '2a9ca18e-4167-4786-9b70-16653e9124fa',
 'c58e9b1d-4db1-47cf-8e81-c68c32cb46e0',
 '1c61931e-1594-423d-867f-c15e88178350',
 '2fe25c99-4818-4be0-ac67-498198f07d85',
 'afa3c9e1-7027-4240-a897-03de94c3cb0f',
 '182cbc32-649b-4217-b113-1b96b37e68af',
 'aa5b7636-0bc2-4ea9-a0e9-96b1ea707f0f',
 'e789c230-9ba7-4173-9dc6-f435c9514b76',
 '99ab26cf-b742-4cdc-b64d-9c8f4964c24f',
 '1b644c29-81eb-

In [22]:
print(vectordb._collection.count())

28


In [23]:
question = "What are major topics for this class?"
docs = vectordb.similarity_search(question,k=3)
len(docs)

3

## RetrievalQA chain

In [24]:
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(model="mistral", temperature=0)


C:\Users\Acer\AppData\Local\Temp\ipykernel_23800\710658220.py:3: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="mistral", temperature=0)


In [25]:
from langchain.chains import RetrievalQA

In [26]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [27]:
result = qa_chain({"query": question})

C:\Users\Acer\AppData\Local\Temp\ipykernel_23800\4094420968.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": question})


In [28]:
result['result']

' Based on the provided context, it appears that the main topics for this class could be:\n\n1. Sign Language Recognition: This includes understanding various sign languages, their gestures, and developing systems to recognize these signs using deep learning techniques such as Convolutional Neural Networks (CNN) and Recurrent Neural Networks (RNN).\n\n2. Deep Learning: This involves studying the principles of deep learning, its applications, and its role in enhancing sign language recognition systems.\n\n3. Computer Vision: This topic focuses on teaching how to process and analyze visual data from a video or image, which is crucial for sign language recognition.\n\n4. Natural Language Processing (NLP): While not explicitly mentioned in the provided context, NLP could be an indirect part of the class given that it\'s often used in conjunction with computer vision for understanding and interpreting signs.\n\n5. Transformer Models: This topic seems to be specifically mentioned as being im

## Prompts

In [29]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)


In [30]:
## run chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever= vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
    
)

In [31]:
question = "how do we find the sing nepali language help to the people and what process are used in that research paper?"
result = qa_chain({"query": question})

In [34]:
result['result']

' The provided research paper utilizes deep learning techniques, specifically Convolutional Neural Networks (CNN) and Recurrent Neural Networks (RNN), for real-time sign language gesture recognition from video sequences. This helps individuals relying on Nepali Sign Language by enhancing communication accessibility. The process involves training the model on a dataset of sign language gestures, which is then used to recognize and interpret signs in real-time.\n\nThanks for asking!'

In [35]:
result['source_documents'][0]

Document(metadata={'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'trapped': '/False', 'creator': 'LaTeX with hyperref', 'title': 'Nepali Sign Language Letter Detection and Finger Spelling Using Mediapipe and CNN', 'producer': 'pdfTeX-1.40.25', 'creationdate': '2024-08-22T12:54:44+05:45', 'page': 3, 'subject': '14th IOE Graduate Conference', 'author': 'Guptaraj Shrestha, Nishan Thing, Prajita Dhakal, Prasanga Dahal, Pukar Karki, Manoj Kumar Guragain', 'keywords': 'Nepali Sign Language, Machine Learning, Convolutional Neural Network, MediaPipe', 'page_label': '169', 'source': 'research.pdf', 'moddate': '2024-08-22T12:54:44+05:45', 'total_pages': 5}, page_content='4 1.00 0.99 0.99 500\n5 1.00 1.00 1.00 500\n6 1.00 1.00 1.00 500\n7 1.00 1.00 1.00 500\n8 1.00 0.99 0.99 500\n9 0.99 0.99 0.99 500\n10 0.98 1.00 0.99 500\n11 0.99 1.00 1.00 500\n12 1.00 1.00 1.00 500\n13 0.99 0.98 0.98 500\n14 0.98 0.98 0.98 500\n15 0.99 0.98 0.99 50

## RetrievalQA chain types

In [40]:
qa_chain_mr = RetrievalQA.from_chain_type(
    chain_type="map_reduce",\
    llm=llm,
    retriever=vectordb.as_retriever()

)

In [42]:
result = qa_chain_mr({"query": question})

c:\langchain_Models\End To End Q&A Chatbot_using_Ollam\chatvenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
c:\langchain_Models\End To End Q&A Chatbot_using_Ollam\chatvenv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Acer\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://hugg

In [43]:
result['result']

' Based on the provided texts, there is no explicit mention of how the Nepali sign language research helps people directly. However, both research papers discuss advancements in recognizing and translating Nepali sign language using deep learning (deep learning) and Convolutional Neural Networks (CNN), respectively. These advancements could potentially aid people who are deaf or hard of hearing by improving communication accessibility.\n\nTo find more information about how these research findings might help people, you may want to read the full texts of the papers if available. They should provide details on the practical applications and potential benefits of their respective methods for Nepali sign language users.'

In [44]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="refine"
)
result = qa_chain_mr({"query": question})
result["result"]

' Given the additional context provided, it appears that the research paper is focused on developing a system for real-time Nepali Sign Language (NSL) gesture recognition using deep learning techniques, specifically Transformer models. The authors have referenced previous works in American Sign Language (ASL) character recognition and Indian Sign Language (ISL) recognition to support their study.\n\nTo help people who use NSL, the process used in this research paper involves collecting video data of NSL gestures, preprocessing the data, and training deep learning models to recognize these gestures. The authors have utilized Transformer models, which outperform other architectures such as Convolutional Neural Networks (CNN) and Recurrent Neural Networks (RNN), in terms of accuracy.\n\nThe research paper aims to develop a system that can accurately recognize NSL gestures in real-time, which could be beneficial for deaf or hard-of-hearing individuals who use NSL as their primary means of 

In [45]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [46]:
question = "Is probability a class topic?"
result = qa_chain({"query": question})
result["result"]

" Based on the provided context, it appears that the focus is primarily on sign language recognition using deep learning techniques such as Convolutional Neural Networks (CNN) and Recurrent Neural Networks (RNN), rather than on probability theory as a class topic. However, probability concepts might be used in the implementation of these deep learning models for tasks like predicting the likelihood of a certain sign language gesture or word being present in the input data. But without explicit mention of probability as a class topic, it's reasonable to assume that it is not the main focus of the provided context."

In [47]:
question = "why are those prerequesites needed?"
result = qa_chain({"query": question})
result["result"]

" The provided context does not explicitly state why the prerequisites (knowledge or skills) are needed, but it can be inferred that they are related to understanding the topic of sign language recognition and the deep learning models used for this purpose.\n\nThe authors of the referenced papers have used deep learning techniques such as Convolutional Neural Networks (CNN), Recurrent Neural Networks (RNN), and Transformers to improve the accuracy of sign language recognition systems. Therefore, having knowledge about these models and their applications would be beneficial for understanding the context provided.\n\nAdditionally, the context mentions a neural network model trained on a dataset with over 2,500 images per label and total of 50 labels. Understanding how to train and evaluate such a model, including concepts like training accuracy, validation accuracy, loss, dropout layer, early stopping function, and epochs, would be important for interpreting the results presented in the 